<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of Data Science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Autoencoders. In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures not your ability to fit a model with high accuracy. 

__*Caution:*__  these approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime locally, on AWS SageMaker, on Colab or on a comparable environment. If something is running longer, double check your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a LSTM classification model
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for object detection
* <a href="#p3">Part 3</a>: Describe a use case for an autoencoder
* <a href="#p4">Part 4</a>: Describe yourself as a Data Science and elucidate your vision of AI

<a id="p1"></a>
## Part 1 - LSTMSs

Use a LSTM to fit a multi-class classification model on Reuters news articles to distinguish topics of articles. The data is already encoded properly for use in a LSTM model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well as the LSTM code we used in class.

__*Note:*__  Focus on getting a running model, not on maxing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [4]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 120.8MB 115kB/s eta 0:00:01   |▊                               | 2.6MB 1.9MB/s eta 0:01:03     |████████████                    | 45.4MB 7.6MB/s eta 0:00:10     |█████████████████▏              | 64.8MB 3.1MB/s eta 0:00:19     |██████████████████████          | 82.7MB 5.2MB/s eta 0:00:08     |███████████████████████▌        | 88.5MB 3.4MB/s eta 0:00:10     |█████████████████████████████   | 109.5MB 9.4MB/s eta 0:00:02
     |████████████████████████████████| 61kB 9.7MB/s  eta 0:00:01
     |████████████████████████████████| 112kB 20.1MB/s eta 0:00:01
     |████████████████████████████████| 450kB 5.3MB/s eta 0:00:01
     |████████████████████████████████| 2.6MB 10.2MB/s eta 0:00:01     |█████▊                          | 471kB 10.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 16.8MB/s eta 0:00:01
     |████████████████████████████████| 3.9MB 495kB/s eta 0:00:011    |▏                               | 20kB 8.7MB/s eta 0:00:01     |█████

In [2]:
import numpy as np

In [19]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
import random
import sys
import os

In [3]:
from tensorflow.keras.datasets import reuters

(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [4]:
# Demo of encoding

word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data")
print(f"London is encoded as {word_index['london']} in the data")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data
London is encoded as 544 in the data
Words are encoded as numbers in our dataset.


In [5]:
# Do not change this line. You need the +1 for some reason. 
max_features = len(word_index.values()) + 1

# TODO - your code!


In [6]:
len(word_index)

30979

In [7]:
data_list = []

for file in word_index:
    if file[-3:] == 'txt':
        with open(f'./articles/{file}', 'r', encoding='utf-8') as f:
            data.append(f.read())

In [8]:
# Encode Data as Chars

# Gather all text 
# Why? 1. See all possible characters 2. For training / splitting later
text = " ".join(word_index)

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [9]:
maxlen = 50
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  50329


In [10]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [11]:
x.shape

(50329, 50, 39)

In [12]:
y.shape

(50329, 39)

In [15]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='nadam')

In [16]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [20]:
model.fit(x, y,
          batch_size=32,
          epochs=10,
          validation_split=.2,
          callbacks=[print_callback])

Train on 40263 samples, validate on 10066 samples
Epoch 1/10
40256/40263 [============================>.] - ETA: 0s - loss: 2.4854
----- Generating text after Epoch: 0
----- Generating with seed: "r's giro requesting living credibly miles entitlem"
r's giro requesting living credibly miles entitlemerg c2trhatr undonaun' inteos ordiming rnflopan arnmricerlial iolingullins kalesh rumure shofreg vinoradad caldowey ihtrncun ig trum binbieitioge weyrerfationo wocbeboranm alcarssis luenks lyesw diyn onattirlen's impiplober'y 3802 65m3 173 225 47a perebe hel unfros instivedtlo pirationgulonss pnat vens anthrring wolng chabericeton pefrer lanilins dart'e mugp's pnenlin enpcimeverc ingulnere iavutol
40263/40263 [==============================] - 108s 3ms/sample - loss: 2.4855 - val_loss: 2.4543
Epoch 2/10
40256/40263 [============================>.] - ETA: 0s - loss: 2.4143
----- Generating text after Epoch: 1
----- Generating with seed: "e bongard lace lach lack perelman stockdraw lacy s"
e bo

## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*

Pad sequences are a function which are used to make all variable length sequences the same length. When running matrix calculations it's important that the shapes of the matrixes are the same in order to allow for proper dot product.

## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*

LSTM allows for the "recall" of past parts of a sequence. Essentially the network configuration allows for memory within the NN.

## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*

RNN and their subset of LSTM are used to generate sequence prediction problems.
These models have proven to be especially useful with text and speech prediction. RNN's on the other hand are not as useful on weather prediction as other models but in theory due to the way that they manage to create predictions on time series they could be useful for weather (although a simple MLP can actually do a better job)

 Examples include: 

Text problems - text prediction such as autocorrect on a cursor line
Speech problems - text translation from video for those unable to hear 
Time series data - weather prediction (note above this is not a great example but I want to stress the time series aspect of answering this question)


<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>

In [24]:
from skimage.io import imread_collection
from skimage.transform import resize #This might be a helpful function for you

images = imread_collection('./frog_images/*.jpg')

In [145]:
image_path = ('/Users/filch/Dropbox/')

In [146]:
PATH = os.path.join(os.path.dirname(image_path))

In [153]:
train_dir = os.path.join(PATH, 'frog_images/training')
validation_dir = os.path.join(PATH, 'frog_images/validation')

In [154]:
PATH

'/Users/filch/Dropbox'

In [155]:
train_dir

'/Users/filch/Dropbox/frog_images/training'

In [156]:
%pwd

'/Users/filch/Dropbox'

In [157]:
validation_dir

'/Users/filch/Dropbox/frog_images/validation'

In [158]:
num_frog_val = len(os.listdir(validation_dir))

In [189]:
num_frog_val

5

In [173]:
total_val = num_frog_val

In [174]:
num_frog_tr = len(os.listdir(train_dir))

In [175]:
total_train = num_frog_tr

print(type(images))
print(type(images[0]), end="\n\n")

print("Each of the Images is a Different Size")
print(images[0].shape)
print(images[1].shape)

Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model. Print out the predictions in any way you see fit. 

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goal* - Check for other things such as fish.

In [176]:
batch_size = 16
epochs = 10
IMG_HEIGHT = 1710
IMG_WIDTH = 1856

In [177]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [193]:
print(train_dir)
print(validation_dir)

/Users/filch/Dropbox/frog_images/training
/Users/filch/Dropbox/frog_images/validation


In [178]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')


Found 0 images belonging to 0 classes.


In [194]:
train_data_gen

In [179]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 0 images belonging to 0 classes.


In [195]:
val_data_gen

In [180]:
import numpy as np
 
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API
 
resnet = ResNet50(weights='imagenet', include_top=False)

In [181]:
for layer in resnet.layers:
    layer.trainable = False

In [182]:
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)

In [183]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [184]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

ValueError: Asked to retrieve element 0, but the Sequence has length 0

<a id="p3"></a>
## Part 3 - Autoencoders

Describe a use case for an autoencoder given that an autoencoder tries to predict its own input. 

__*Your Answer:*__ 

An autoencoders base usage is for dimensionality reduction. This can be used effectively to reduce noise in images. 


<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?

I have a strong understanding of the concepts involved and have a decades long experience with business models. My strongest suit will be using data science to bear on business problems.

- What area of Data Science would you most like to learn more about, and why?

I am extremely interested in artifical intelligence. I began working on this in high school and continued into college. I'm fascinated by how the tools have changed since the 80's and early 90's when I last spent siginificant parts of my life exploring this topic.

- Where do you think Data Science will be in 5 years?

I think there is a good chance we will be at the singularity within the next decade.

- What are the threats posed by AI to our society?

I think our biggest threat is when/if the GAI is able to take root in the internet. As long as it can be contained within standalone devices I'm reasonably sure we will be fine into the foreseeable future.

- How do you think we can counteract those threats? 

This is something I think about a lot. One way is to ensure GAI is tied to specific devices and not networks.

- Do you think achieving General Artifical Intelligence is ever possible?

Yes. It will be here soon.

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [ ]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")